In [7]:
from aniso_MLMD.trainer import load_datasets
import aniso_MLMD.model.neighbor_ops as neighbor_ops
import torch

In [2]:
data_path ="/home/marjan/Documents/code-base/ml_datasets/pps_20_new/"

In [3]:
train_df = pd.read_pickle(os.path.join(data_path, 'train.pkl'))

In [4]:
train_dataloader, valid_dataloader, test_dataloader = load_datasets(data_path, 5)

In [31]:
for i, ((position, q, orientation_R, neighbor_list), target_force, target_torque,
                energy) in enumerate(
                train_dataloader):
    break

In [32]:
box_len = 16.17887

In [33]:
neighbor_list.shape

torch.Size([5, 200, 2])

In [34]:
batch_size = position.shape[0]
N_particles = position.shape[1]

# change tuple based neighbor list to (B, N, neighbor_idx)
neighbor_list = neighbor_list.reshape(batch_size, N_particles, -1,
                                      neighbor_list.shape[-1])[:, :, :,
                1]  # (B, N, N_neighbors)
N_neighbors = neighbor_list.shape[-1]
dr = neighbor_ops.neighbors_distance_vector(position,
                                            neighbor_list)  # (B, N, N_neighbors, 3)
dr = neighbor_ops.adjust_periodic_boundary(dr, box_len)

R = torch.norm(dr, dim=-1, keepdim=True)  # (B, N, N_neighbors, 1)

inv_R = 1. / R  # (B, N, N_neighbors, 1)

################ orientation related features ################

# repeat the neighbors idx to match the shape of orientation_R. This
# is necessary to gather each particle's neighbors' orientation
NN_expanded = neighbor_list[:, :, :, None, None].expand(
    (-1, -1, -1, 3, 3))  # (B, N, N_neighbors, 3, 3)
# repeart the orientation_R to match the shape of neighbor_list
orientation_R_expanded = orientation_R[:, :, None, :, :].expand(
    (-1, -1, N_neighbors, -1, -1))  # (B, N, N_neighbors, 3, 3)
# get all neighbors' orientation
neighbors_orient_R = torch.gather(orientation_R_expanded, dim=1,
                                  index=NN_expanded)  # (B, N, N_neighbors, 3, 3)

# dot product: (B, N, N_neighbors, 3, 3)
orient_dot_prod = neighbor_ops.orientation_dot_product(
    orientation_R_expanded,
    neighbors_orient_R)

# element product: (B, N, N_neighbors, 3, 3, 3)
print('here')
orient_element_prod = neighbor_ops.orientation_element_product(
    orientation_R_expanded,
    neighbors_orient_R)
print(orient_element_prod.shape)
# element product norm: (B, N, N_neighbors, 3, 3)
element_prod_norm = torch.norm(orient_element_prod,
                               dim=-1)

# principal cross product: (B, N, N_neighbors, 3, 3)
orient_cross_prod = neighbor_ops.orientation_principal_cross_product(
    orientation_R_expanded,
    neighbors_orient_R)
# cross product norm: (B, N, N_neighbors, 3)
cross_prod_norm = torch.norm(orient_cross_prod,
                             dim=-1)

# relative orientation: (B, N, N_neighbors, 3, 3)
rel_orient = neighbor_ops.relative_orientation(orientation_R_expanded,
                                               neighbors_orient_R)

################ RBF features ################

# RBF for particles:(B, N, N_neighbors, 3)
rbf_particle = neighbor_ops.RBF_dr_orientation(dr,
                                               orientation_R_expanded)
# RBF for neighbors: (B, N, N_neighbors, 3)
rbf_neighbors = neighbor_ops.RBF_dr_orientation(dr, neighbors_orient_R)

# euler angle (B, N, N_neighbors, 3)
angle = neighbor_ops.rot_matrix_to_euler_angle(rel_orient)

here
torch.Size([5, 20, 10, 3, 3, 3])


In [10]:
R.shape

torch.Size([5, 20, 10, 1])

In [43]:
features = torch.cat((R,
                      dr / R,
                      inv_R,
                      orient_dot_prod.flatten(start_dim=-2),
                      orient_element_prod.flatten(start_dim=-3),
                      element_prod_norm.flatten(start_dim=-2),
                      orient_cross_prod.flatten(start_dim=-2),
                      cross_prod_norm,
                      rel_orient.flatten(start_dim=-2),
                      # rbf_particle,
                      # rbf_neighbors,
                      # angle
                      ),
                     dim=-1)


In [44]:
features.shape

torch.Size([5, 20, 10, 71])

In [30]:
neighbor_list.shape

torch.Size([5, 20, 1])

In [28]:
orient_element_prod = neighbor_ops.orientation_element_product(
    orientation_R_expanded,
    neighbors_orient_R)

KeyboardInterrupt: 